Q1

In [1]:
import pandas as pd
from gurobipy import Model, GRB
import numpy as np

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_33423/3415466315.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('/Users/pratiksha/Downloads/price_response.csv')

In [3]:
model = Model('Maximize Revenue')

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
# Variables
prices = model.addVars(df.index, name='price', lb=0)  # Price variables for each product

In [5]:
# Define the objective function (total revenue)
revenue = sum((df.loc[i, 'Intercept'] + df.loc[i, 'Sensitivity'] * prices[i]) * prices[i] for i in df.index)
model.setObjective(revenue, GRB.MAXIMIZE)

In [6]:
# Add capacity constraints
for i in df.index:
    demand = df.loc[i, 'Intercept'] + df.loc[i, 'Sensitivity'] * prices[i]
    model.addConstr(demand <= df.loc[i, 'Capacity'], name=f'capacity_{i}')

In [7]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 9 columns and 9 nonzeros
Model fingerprint: 0xd1ce905f
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 9 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective 7.38368672e+08


In [8]:
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for i in df.index:
        print(f"{df.loc[i, 'Product']}: ${prices[i].X:.2f}")
    print(f"Total Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution found.")

Optimal Prices:
Line 1 Product 1: $383.85
Line 1 Product 2: $2296.50
Line 1 Product 3: $2351.88
Line 2 Product 1: $2050.30
Line 2 Product 2: $4160.71
Line 2 Product 3: $6813.66
Line 3 Product 1: $8138.45
Line 3 Product 2: $4498.42
Line 3 Product 3: $8558.94
Total Revenue: $738368671.96
